In [ ]:
# === Код для вставки в ячейку Jupyter Notebook ===

import pandas as pd
import re
from datetime import datetime
# import requests # Обычно не требуется напрямую для pd.read_html

print("--- Начало скрипта ---")

# URL страницы Википедии со списком компаний S&P 500
URL_WIKIPEDIA_SP500 = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

print(f"Попытка загрузить таблицы с: {URL_WIKIPEDIA_SP500}")

try:
    # Загружаем все таблицы с HTML-страницы
    # pandas.read_html требует наличия одного из парсеров: lxml, html5lib, или beautifulsoup4
    # Убедитесь, что они установлены: !pip install lxml html5lib beautifulsoup4
    tables = pd.read_html(URL_WIKIPEDIA_SP500)
    
    if not tables:
        print("На странице не найдено таблиц.")
    else:
        print(f"Найдено {len(tables)} таблиц. Предполагаем, что первая таблица содержит список S&P 500.")
        
        # Обычно первая таблица на странице - это список компонентов S&P 500
        sp500_df_original = tables[0].copy() # Используем .copy() для работы с копией
        
        print("\nСтолбцы в загруженной таблице:")
        print(sp500_df_original.columns)
        
        # --- Шаг 1: Создание DataFrame с тикерами, именами и годом добавления ---
        
        # Определение имен столбцов (могут немного отличаться на Википедии)
        symbol_col = 'Symbol'
        name_col = 'Security' # Название компании
        
        # Поиск столбца с датой добавления
        date_added_col = None
        possible_date_cols = ['Date added', 'Date first added', 'First added'] # Возможные названия столбца
        for col_name in possible_date_cols:
            if col_name in sp500_df_original.columns:
                date_added_col = col_name
                break
        
        if not date_added_col:
            # Если столбец не найден с ожидаемыми именами, можно попробовать найти его по ключевым словам
            # или вывести ошибку с просьбой проверить вручную
            print(f"ПРЕДУПРЕЖДЕНИЕ: Не удалось автоматически найти столбец с датой добавления по именам: {possible_date_cols}.")
            print(f"Пожалуйста, проверьте столбцы: {sp500_df_original.columns} и при необходимости измените 'date_added_col' вручную в коде.")
            # В качестве запасного варианта, можно попробовать взять столбец, если он содержит слово 'Date' или 'Added'
            # Это менее надежно. Для простоты, если не найден, выведем ошибку позже.
            # Попытка найти столбец, если точного совпадения нет (менее надежно)
            for col in sp500_df_original.columns:
                if 'date' in col.lower() or 'added' in col.lower():
                    if sp500_df_original[col].astype(str).str.contains(r'\d{4}', na=False).any(): # Проверяем, есть ли в нем что-то похожее на год
                        date_added_col = col
                        print(f"Используется предполагаемый столбец с датой: '{date_added_col}'")
                        break
            if not date_added_col:
                 raise ValueError(f"Не удалось найти столбец с датой добавления. Проверьте столбцы: {sp500_df_original.columns}")


        print(f"\nИспользуемые столбцы: Тикер='{symbol_col}', Имя='{name_col}', Дата добавления='{date_added_col}'")

        # Проверка наличия ключевых столбцов
        if symbol_col not in sp500_df_original.columns:
            raise ValueError(f"Столбец тикера '{symbol_col}' не найден. Доступные столбцы: {sp500_df_original.columns}")
        if name_col not in sp500_df_original.columns:
             # Если нет 'Security', попробуем 'Name' или первый столбец после тикера
            if 'Name' in sp500_df_original.columns:
                name_col = 'Name'
            else: # крайний случай
                print(f"Столбец '{name_col}' (Security) не найден, пытаемся использовать следующий после тикера как имя.")
                symbol_idx = sp500_df_original.columns.get_loc(symbol_col)
                if symbol_idx + 1 < len(sp500_df_original.columns):
                    name_col = sp500_df_original.columns[symbol_idx+1]
                else:
                    raise ValueError(f"Не удалось определить столбец с именем компании.")
            print(f"Обновленный столбец имени компании: '{name_col}'")


        # Выбор нужных столбцов
        df_companies = sp500_df_original[[symbol_col, name_col, date_added_col]].copy()
        df_companies.rename(columns={
            symbol_col: 'Ticker',
            name_col: 'Name',
            date_added_col: 'DateAddedRaw'
        }, inplace=True)

        # Функция для извлечения года из строки с датой
        def extract_year_from_date(date_entry):
            if pd.isna(date_entry):
                return None
            date_str = str(date_entry)
            
            # Попытка прямого преобразования в datetime (предпочтительно)
            try:
                # to_datetime может обработать много стандартных форматов
                dt_object = pd.to_datetime(date_str, errors='coerce') # 'coerce' вернет NaT при ошибке
                if pd.notna(dt_object):
                    return dt_object.year
            except Exception: 
                pass # Если не получилось, пробуем регулярное выражение ниже
            
            # Если to_datetime не справился, используем регулярное выражение
            # для поиска 4-значного года (например, "YYYY", "Month Day, YYYY", "YYYY[footnote]")
            match = re.search(r'(\b\d{4}\b)', date_str)
            if match:
                return int(match.group(1))
            
            return None # Возвращаем None, если год извлечь не удалось

        df_companies['YearAdded'] = df_companies['DateAddedRaw'].apply(extract_year_from_date)
        
        # Удаляем строки, где год извлечь не удалось (критично для анализа)
        initial_rows = len(df_companies)
        df_companies.dropna(subset=['YearAdded'], inplace=True)
        cleaned_rows = len(df_companies)
        print(f"\nУдалено {initial_rows - cleaned_rows} строк из-за невозможности извлечь год добавления.")
        
        if df_companies.empty:
            raise ValueError("После очистки не осталось данных для анализа. Проверьте столбец с датами.")

        df_companies.loc[:, 'YearAdded'] = df_companies['YearAdded'].astype(int)


        print("\nDataFrame с извлеченными годами (первые 5 строк):")
        print(df_companies.head())

        # --- Шаг 2: Извлечение года и подсчет количества акций, добавленных каждый год ---
        additions_per_year = df_companies.groupby('YearAdded').size().reset_index(name='NumberOfAdditions')
        
        print("\nКоличество акций (из текущего списка S&P 500), добавленных по годам (топ 10 по количеству):")
        print(additions_per_year.sort_values(by='NumberOfAdditions', ascending=False).head(10))

        # --- Шаг 3: Определение года с наибольшим количеством добавлений (исключая 1957) ---
        # Исключаем 1957 год, так как это год основания индекса в его текущем формате
        additions_filtered = additions_per_year[additions_per_year['YearAdded'] != 1957].copy() # .copy() чтобы избежать SettingWithCopyWarning
        
        if not additions_filtered.empty:
            max_additions_count = additions_filtered['NumberOfAdditions'].max()
            # Находим все годы с этим максимальным количеством
            years_with_max_additions_df = additions_filtered[additions_filtered['NumberOfAdditions'] == max_additions_count]
            
            # Если несколько лет с одинаковым максимумом, выбираем самый последний (наибольший год)
            year_highest_additions = years_with_max_additions_df['YearAdded'].max()
            
            print(f"\n--------------------------------------------------------------------")
            print(f"Вопрос 1: Год с наибольшим количеством добавлений (исключая 1957): {year_highest_additions}")
            print(f"           (Количество добавлений в этот год для {year_highest_additions}: {max_additions_count})")
            print(f"--------------------------------------------------------------------")
        else:
            print("\nНе удалось определить год с наибольшим количеством добавлений после фильтрации 1957 года.")

        # --- Дополнительный вопрос: Сколько текущих акций S&P 500 находятся в индексе более 20 лет? ---
        current_processing_year = datetime.now().year # Год, когда запускается скрипт
        # "Более 20 лет" означает, что компания была добавлена 21 год назад или ранее
        # Например, если сейчас 2024:
        # Добавлена в 2003 -> 2024 - 2003 = 21 год (это "более 20 лет")
        # Добавлена в 2004 -> 2024 - 2004 = 20 лет (это "20 лет", а не "более 20 лет")
        # Значит, год добавления должен быть <= current_processing_year - 21
        threshold_year = current_processing_year - 21 

        stocks_over_20_years_df = df_companies[df_companies['YearAdded'] <= threshold_year].copy()
        count_stocks_over_20_years = len(stocks_over_20_years_df)
        
        print(f"\n--------------------------------------------------------------------")
        print(f"Дополнительно: Количество текущих акций S&P 500, находящихся в индексе")
        print(f"              более 20 лет (т.е. добавлены в {threshold_year} году или ранее, исходя из текущего года {current_processing_year}):")
        print(f"              Результат: {count_stocks_over_20_years} компаний.")
        print(f"--------------------------------------------------------------------")
        
        if not stocks_over_20_years_df.empty:
            print("\nПримеры акций, находящихся в индексе более 20 лет (первые 5, если есть):")
            print(stocks_over_20_years_df[['Ticker', 'Name', 'YearAdded']].head())
        else:
            print("\nНе найдено акций, находящихся в индексе более 20 лет (согласно текущим данным и критерию).")


# Обработка возможных ошибок при выполнении
except ImportError as e:
    print(f"\nОшибка импорта: {e}.")
    print("Пожалуйста, убедитесь, что все необходимые библиотеки установлены.")
    print("Вы можете установить их, выполнив в ячейке Jupyter: !pip install pandas lxml html5lib beautifulsoup4")
# except requests.exceptions.RequestException as e: # pd.read_html может вызывать свои ошибки сети
#     print(f"\nСетевая ошибка при попытке доступа к {URL_WIKIPEDIA_SP500}: {e}")
#     print("Пожалуйста, проверьте ваше интернет-соединение.")
except ValueError as e: # Ошибки, связанные с данными или структурой таблицы
    print(f"\nОшибка значения или обработки данных: {e}")
    print("Это может быть связано с изменениями в структуре таблицы на Википедии или отсутствием ожидаемых столбцов.")
except Exception as e: # Любые другие непредвиденные ошибки
    print(f"\nПроизошла неожиданная ошибка: {e}")
    print("Возможные причины:")
    print("- Проблемы с интернет-соединением, не перехваченные выше.")
    print("- Значительные изменения в структуре страницы Википедии.")
    print("- Другие проблемы с данными или кодом.")

print("\n--- Конец скрипта ---")